In [182]:
# Проверка возможности использования xgbooster в основном, либо предварительно (2017-06-30)

In [183]:
import sys,os,datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import fbeta_score
from __future__ import print_function

In [184]:
print(np.__version__);
print(pd.__version__);

1.12.1
0.19.2


In [185]:
import  cv2 as cv
cv.__version__

'3.2.0'

In [186]:
sys.path.append('../Python')
from helper import formFH, paths_input

In [198]:
binsData = 75

trLabels,trDirTIF,trDirJPG,teDirTIF,teDirJPG = paths_input()
if True :
    trDirI = trDirJPG
    teDirI = teDirJPG
    ttExt  = '.jpg'
else :
    trDirI = trDirTIF
    teDirI = teDirTIF
    ttExt  = '.tif'
trWork, teWork = '../Work/Train', '../Work/Test'
print (trDirI,teDirI, trWork, teWork)
print (trLabels,trDirTIF,trDirJPG,teDirTIF,teDirJPG)

../Data/train-jpg ../Data/test-jpg-v2 ../Work/Train ../Work/Test
../Data/train_v2.csv ../Data/train-tif-v2 ../Data/train-jpg ../Data/test-tif-v2 ../Data/test-jpg-v2


In [188]:
xx=os.listdir(trDirI); xx.sort(); xx[:6]

['train_0.jpg',
 'train_1.jpg',
 'train_10.jpg',
 'train_100.jpg',
 'train_1000.jpg',
 'train_10000.jpg']

In [189]:
labels_df = pd.read_csv(trLabels)
labels_df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [190]:
# Build list with unique labels
label_list = []
for tag_str in labels_df.tags.values:
    labels = tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

In [191]:
# Add features (0 or 1) for every label as dataframe field 
for label in label_list:
    labels_df[label] = labels_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
#labels_df.head()

In [192]:
weather_labels = ['clear', 'partly_cloudy', 'haze', 'cloudy']
land_labels = ['primary', 'agriculture', 'water', 'cultivation', 'habitation' ]
rare_labels = [l for l in label_list if labels_df[label_list].sum()[l] < 2000]
#rare_labels              

In [193]:
labels = label_list; #weather_labels;
nameList =labels_df[labels_df[labels].sum(axis=1)>0].image_name.tolist(); len(nameList)
labelList=labels_df[labels_df[labels].sum(axis=1)>0][labels].as_matrix();
labelList[:6,:]
#labels_df.head()

array([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [194]:
def formImExt (nf, resize=(32,32), printOK=False) :
    nx = None
    try : 
        ni = cv.imread(nf,-1); 
        if (ni is not None) :
            if not ((ni.shape[2]==3) or (ni.shape[2]==4)) and printOK : print('----- error ---- shape:',ni.shape,nf)
            if (ni.shape[2]==3) :
                nx = cv.resize(ni,resize)
            if (ni.shape[2]==4) :
                #r,g,b,n = ni[:,:,2],ni[:,:,1],ni[:,:,0],ni[:,:,3]
                r,g,b,n = cv.resize(ni[:,:,2],resize),cv.resize(ni[:,:,1],resize),cv.resize(ni[:,:,0],resize),cv.resize(ni[:,:,3],resize)
                dv,dw   = np.divide((r-n),(r+n+0.01)), np.divide((g-n),(g+n+0.01))
                nx      = np.array([r,g,b,n,dv,dw]).T; 
    except BaseException as e :
        print(nf,e); nx = None;
    
    if nx is None and printOK : 
        print('------ None:',nf); nx = None
        
    return(nx)

def formImHist (nf, size, printOK=False) :
    
    def hist1 (nf1,bins) :
        h,_ = np.histogram(nf1.ravel(),bins=bins)
        return h.tolist()
    
    def histN (nf,bins) :
        h = []
        for i in range(0,nf.shape[2]) :
            h = h + hist1(nf[:,:,i],bins=bins);
        return h
             
    nx = None
    try : 
        ni = cv.imread(nf,-1);
        if (ni is not None) :
            
            if printOK : print(nf,ni.shape)

            if not ((ni.shape[2]==3) or (ni.shape[2]==4)) and printOK : 
                print('----- error ---- shape:',ni.shape,nf);
            if  ni.shape[2]==3 :
                bins = [x for x in range(0,255+int(round(256.0/size)),int(round(256.0/size)))]
                nx = histN(ni,bins)
            if  ni.shape[2]==4 : 
                bins = [x for x in range(0,65535+int(round(65535.0/size)),int(round(65535.0/size)))]
                nx = histN(ni,bins)
            
    except BaseException as e :
        print(nf,e); nx = None;
    
    if nx is None and printOK : 
        print('------ None:',nf); nx = None
        
    return(nx)

In [195]:
os.path.exists(nf)
ni = cv.imread(nf,-1);
ni.shape

(256, 256, 3)

In [199]:
trX, trY, i, size = [],[], 0, len(nameList)
print(datetime.datetime.now(),len(nameList),size)
for nn in nameList[0:size] :
    nf = os.path.join(trDirI,nn+".jpg");
    nx = formImHist(nf,binsData,printOK=False)
    if (nx is not None) :
        #rr=np.save(os.path.join(trWork,nn+".npy"),nx);
        #trX.append(nn+".npy")
        trX.append(nx)
        trY.append(True)
    else : 
        trY.append(False)
    i += 1
    if (i%5000==0) : print(datetime.datetime.now(),"\t",i,"\t",nn)
    #print(nn.shape)

print(datetime.datetime.now())
trX = np.array(trX);
trY = labelList[trY];
print(len(nameList),trX.shape,trY.shape)
#trXY=pd.DataFrame(trY); trXY['name']=trX; trXY.head()
#trXY.to_pickle(os.path.join(trWork,"listFiles.pkl"))

2017-07-01 04:51:04.149953 40479 40479
2017-07-01 04:51:36.674857 	 5000 	 train_4999
2017-07-01 04:52:08.543257 	 10000 	 train_9999
2017-07-01 04:52:40.446697 	 15000 	 train_14999
2017-07-01 04:53:12.665935 	 20000 	 train_19999
2017-07-01 04:53:43.842776 	 25000 	 train_24999
2017-07-01 04:54:16.205125 	 30000 	 train_29999
2017-07-01 04:54:48.352807 	 35000 	 train_34999
2017-07-01 04:55:20.339310 	 40000 	 train_39999
2017-07-01 04:55:23.410228
40479 (40479, 255) (40479, 17)


In [200]:
trX.shape,trY.shape

((40479, 255), (40479, 17))

In [201]:
np.save('../Work/train-data-XX.npy',trX)
np.save('../Work/train-data-YY.npy',trY)

In [202]:
del(trX); del(trY);

In [203]:
trX = np.load('../Work/train-data-XX.npy')
trY = np.load('../Work/train-data-YY.npy')
print(trX.shape,trY.shape)

(40479, 255) (40479, 17)


In [204]:
import xgboost as xgb

In [205]:
# Строим основные массивы
dtrain = xgb.DMatrix(trX,label=trY[:,1])

In [206]:
param = {
         'objective':'reg:logistic', 
         #'num_class':17,
         #'eta':0.05, 
         #'max_depth':9, 'subsample':0.8, 'colsample_bytree':0.8, 
         #'metrics':'rmse'
         'eval_metric':['auc','rmse']
        }
param['silent']=True

results, bstList = [], []
for i in range(0,trY.shape[1]) :
    print ('Working: {}'.format(i))
    dtrain.set_label(trY[:,i])
    history = xgb.cv(param, dtrain, metrics=['auc'], verbose_eval=30, num_boost_round=1000, nfold=3, early_stopping_rounds=20) #,stratified=True,maximize=True)
    results.append(history)
    rvalidCV = {}
    watchlistCV  = [(dtrain,'train')]
    bst = xgb.train(param, dtrain, history['test-auc-mean'].argmax()+1, watchlistCV) 
    bstList.append(bst)
    

Working: 0
[0]	train-auc:0.948523+0.000546038	test-auc:0.933832+0.00678
[30]	train-auc:0.988173+0.00141962	test-auc:0.959638+0.00215308
[0]	train-auc:0.951655	train-rmse:0.377481
[1]	train-auc:0.957275	train-rmse:0.300147
[2]	train-auc:0.9596	train-rmse:0.250336
[3]	train-auc:0.961825	train-rmse:0.218725
[4]	train-auc:0.963702	train-rmse:0.198916
[5]	train-auc:0.966345	train-rmse:0.187646
[6]	train-auc:0.968926	train-rmse:0.181068
[7]	train-auc:0.969649	train-rmse:0.176983
[8]	train-auc:0.971442	train-rmse:0.174396
[9]	train-auc:0.972018	train-rmse:0.172895
[10]	train-auc:0.973012	train-rmse:0.171047
[11]	train-auc:0.974368	train-rmse:0.170097
[12]	train-auc:0.975208	train-rmse:0.169154
[13]	train-auc:0.976176	train-rmse:0.168202
[14]	train-auc:0.976391	train-rmse:0.167754
[15]	train-auc:0.977798	train-rmse:0.166445
[16]	train-auc:0.978714	train-rmse:0.164622
[17]	train-auc:0.979372	train-rmse:0.164034
[18]	train-auc:0.979922	train-rmse:0.163545
[19]	train-auc:0.980503	train-rmse:0.162

In [112]:
#results[0]

In [113]:
#[(x['test-rmse-mean'].min(),x['test-rmse-mean'].argmin()) for x in results]

In [207]:
#[(x['test-rmse-mean'].min(),x['test-rmse-mean'].argmin()) for x in results]

In [161]:
[(x['test-auc-mean'].max(),x['test-auc-mean'].argmax()) for x in results]

[(0.95933766666666676, 51),
 (0.97132033333333334, 70),
 (0.92636800000000008, 84),
 (0.97527066666666673, 52),
 (0.82370966666666667, 91),
 (0.93564866666666668, 41),
 (0.91020533333333331, 58),
 (0.84959400000000007, 84),
 (0.84758433333333338, 38),
 (0.98782933333333334, 197),
 (0.9787530000000001, 214),
 (0.95955733333333326, 49),
 (0.86465266666666662, 54),
 (0.98490033333333338, 55),
 (0.88952866666666663, 32),
 (0.87565166666666672, 27),
 (0.81941499999999989, 40)]

In [208]:
[(x['test-auc-mean'].max(),x['test-auc-mean'].argmax()) for x in results]

[(0.95963833333333337, 30),
 (0.97144833333333336, 73),
 (0.92635233333333333, 109),
 (0.97506933333333334, 130),
 (0.82618333333333327, 151),
 (0.93428500000000003, 56),
 (0.91021033333333345, 56),
 (0.84739166666666665, 74),
 (0.84609366666666663, 31),
 (0.98742033333333323, 82),
 (0.97728066666666669, 190),
 (0.957928, 27),
 (0.86184966666666674, 40),
 (0.98402533333333331, 56),
 (0.88504333333333329, 28),
 (0.87253733333333328, 39),
 (0.83189766666666676, 36)]

In [255]:
# Forming output dataset for predicting --> trOX, trOY
del(trX)
del(trY)

In [209]:
nameAsk = os.listdir(teDirI); print(len(nameAsk))
trOX, trOY, i, size = [], [], 0, len(nameAsk)
print(datetime.datetime.now(),len(nameAsk),size)
for nn in nameAsk[0:size] :
    nf = os.path.join(teDirI,nn);
    nx = formImHist(nf,binsData) #,printOK=True)
    if (nx is not None) :
        trOX.append(nx)
        trOY.append(nn)
    i += 1
    if (i%5000==0) : print(datetime.datetime.now(),"\t",i,"\t",nn)

print(datetime.datetime.now())

#assert (size!=len(trOY)), "Wrong files {} != {}".format(size,len(trOY))

61191
2017-07-01 05:06:01.930043 40479 61191
2017-07-01 05:06:34.479920 	 5000 	 test_14523.jpg
2017-07-01 05:07:06.824202 	 10000 	 test_19029.jpg
2017-07-01 05:07:39.460715 	 15000 	 test_23524.jpg
2017-07-01 05:08:11.786825 	 20000 	 test_28015.jpg
2017-07-01 05:08:44.104220 	 25000 	 test_32520.jpg
2017-07-01 05:09:16.153673 	 30000 	 test_37026.jpg
2017-07-01 05:09:48.651442 	 35000 	 test_4908.jpg
2017-07-01 05:10:21.055375 	 40000 	 test_9402.jpg
2017-07-01 05:10:53.606226 	 45000 	 file_13913.jpg
2017-07-01 05:11:25.660329 	 50000 	 file_18419.jpg
2017-07-01 05:11:57.644550 	 55000 	 file_4564.jpg
2017-07-01 05:12:29.440483 	 60000 	 file_892.jpg
2017-07-01 05:12:37.113322


In [210]:
trOX = np.array(trOX);
trOY = np.array([os.path.splitext(x)[0] for x in trOY]);
print(len(nameAsk),trOX.shape,trOY.shape)

61191 (61191, 255) (61191,)


In [211]:
#trOX[20],trOY[20]

In [212]:
# Saving & Loading

In [213]:
np.save('../Work/test-data-XX.npy',trOX)
np.save('../Work/test-data-YY.npy',trOY)

In [214]:
trOX = np.load('../Work/test-data-XX.npy')
trOY = np.load('../Work/test-data-YY.npy')
print(trOX.shape,trOY.shape)

(61191, 255) (61191,)


In [215]:
# Формирование результата
dtest  = xgb.DMatrix(trOX)

In [216]:
trP = []
for bstI in bstList :
    trP1 = bstI.predict(dtest);
    trP.append(trP1)
trP = np.array(trP).T
type(trP),trP.shape    

(numpy.ndarray, (61191, 17))

In [217]:
np.save('../Work/test-data-RR.npy',trP)

In [218]:
trP = np.load('../Work/test-data-RR.npy')
trP.shape

(61191, 17)

In [219]:
wr = [labels.index(i) for i in weather_labels];
trM=np.array([0.5]*17);
trM[np.array(wr)] = 2.0
print(wr,'\n',labels,'\n',trM.tolist())

[3, 10, 0, 9] 
 ['haze', 'primary', 'agriculture', 'clear', 'water', 'habitation', 'road', 'cultivation', 'slash_burn', 'cloudy', 'partly_cloudy', 'conventional_mine', 'bare_ground', 'artisinal_mine', 'blooming', 'selective_logging', 'blow_down'] 
 [2.0, 0.5, 0.5, 2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 2.0, 2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


In [220]:
#trP = model.predict(trX, batch_size=512); trP=K.get_value(trP)
res = []

for i in range(trP.shape[0]) :
    trA1 = [weather_labels[trP[i,wr].argmax()]]
    trA2 = [labels[ii] for ii in range(len(labels)) if (trP[i,ii]>trM[ii])]
    trPP = trA1 + trA2;
    pp   = ' '.join(trPP)
    ##if (pp=="") : print(trY[i])
    res.append([trOY[i],pp])

res.sort(cmp=lambda x,y: cmp(int(x[0].partition('_')[2]),int(y[0].partition('_')[2])) if (x[0].partition('_')[0]==y[0].partition('_')[0]) else cmp(y[0].partition('_')[0],x[0].partition('_')[0]))
#print(res[4:8],'\n',res[-4:])
print(res[0],'\n',res[-1],'\n',trM,trP[0])

['test_0', 'clear primary'] 
 ['file_20521', 'clear primary'] 
 [ 2.   0.5  0.5  2.   0.5  0.5  0.5  0.5  0.5  2.   2.   0.5  0.5  0.5  0.5
  0.5  0.5] [  1.10700212e-01   2.90093482e-01   2.22463477e-02   2.27843031e-01
   6.58276916e-01   1.19877968e-03   1.15448106e-02   6.40173617e-04
   2.41632879e-04   8.03460598e-01   1.32698828e-04   2.17054214e-04
   6.70437142e-03   1.04527144e-05   2.48264405e-04   4.32673929e-04
   6.35585457e-05]


In [221]:
print(labels)
print(trM.tolist())
np.round(trP[4:11,:])

['haze', 'primary', 'agriculture', 'clear', 'water', 'habitation', 'road', 'cultivation', 'slash_burn', 'cloudy', 'partly_cloudy', 'conventional_mine', 'bare_ground', 'artisinal_mine', 'blooming', 'selective_logging', 'blow_down']
[2.0, 0.5, 0.5, 2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 2.0, 2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


array([[ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.]], dtype=float32)

In [222]:
rrr=pd.DataFrame(res,columns=['image_name','tags']); rrr.head(); 
suffixDT = (datetime.datetime.now()).strftime('%Y-%m-%d-%H-%M-%S'); print(suffixDT)
rrr.to_csv('../Result/vss'+suffixDT+'.csv',index=False);

2017-07-01-05-14-03
